# 🤖 Compliance Agent - Guia de Uso para Desenvolvedores

Este notebook demonstra como usar o **Compliance Agent** refatorado com **State Design Pattern**.

## 📋 Índice

1. [Visão Geral da Arquitetura](#visao-geral)
2. [State Design Pattern](#state-pattern)
3. [Configuração Inicial](#configuracao)
4. [Exemplo Básico de Uso](#exemplo-basico)
5. [Entendendo o Fluxo de Estados](#fluxo-estados)
6. [Interpretando Resultados do ISR](#interpretando-resultados)
7. [Exemplos Avançados](#exemplos-avancados)
8. [Boas Práticas](#boas-praticas)

---

## 🎯 Objetivo

O Compliance Agent é um sistema que:
- ✅ Audita decisões usando ISR (Information Sufficiency Ratio)
- ✅ Detecta alucinações em respostas de LLMs
- ✅ Verifica se decisões são suportadas por contexto
- ✅ Usa State Pattern para eliminar lógica condicional frágil

## 🏗️ Visão Geral da Arquitetura {#visao-geral}

### Estrutura do Projeto

```
src/
├── agent.py              # ComplianceAgent (Context do State Pattern)
├── tools/
│   └── isr_auditor.py    # SemanticISRAuditorTool (lógica matemática ISR)
└── states/
    ├── base.py           # AgentState (interface abstrata)
    ├── idle.py           # Estado inicial
    ├── analysis.py       # Análise com LLM
    ├── audit.py          # Execução do ISR audit
    ├── final_response.py # Geração da resposta final
    └── error.py          # Tratamento de erros
```

### Componentes Principais

1. **ComplianceAgent** (Context): Mantém estado, histórico e ferramentas
2. **AgentState** (Interface): Define o contrato para todos os estados
3. **Estados Concretos**: Implementam a lógica específica de cada fase
4. **SemanticISRAuditorTool**: Executa a matemática do ISR v3

## 🔄 State Design Pattern {#state-pattern}

### O que é o State Pattern?

O **State Pattern** é um padrão de design que permite a um objeto alterar seu comportamento quando seu estado interno muda. Isso elimina a necessidade de múltiplas condicionais (`if/else`) espalhadas pelo código.

### Fluxo do State Machine

```
┌─────────────┐
│ IdleState   │ ← Estado inicial, captura query do usuário
└──────┬──────┘
       │
       ↓
┌─────────────┐
│AnalysisState│ ← Analisa intenção, chama LLM, extrai tool_call
└──────┬──────┘
       │
       ↓
┌─────────────┐
│ AuditState  │ ← Executa ISR audit (matemática completa)
└──────┬──────┘
       │
       ↓
┌─────────────┐
│FinalResponse│ ← Gera resposta final ao usuário
└──────┬──────┘
       │
       ↓
┌─────────────┐
│ IdleState   │ ← Retorna ao início (pronto para próxima query)
└─────────────┘
```

### Vantagens

✅ **Sem lógica condicional frágil**: Não há `if msg.tool_calls...` espalhados  
✅ **Fácil de estender**: Adicionar novos estados é simples  
✅ **Testável**: Cada estado pode ser testado isoladamente  
✅ **Manutenível**: Lógica clara e organizada por responsabilidade

## ⚙️ Configuração Inicial {#configuracao}

In [ ]:
# Importações necessárias
import os
import sys
from dotenv import load_dotenv
from openai import OpenAI

# Carregar variáveis de ambiente
load_dotenv()

# Verificar se a API key está configurada
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

print("✅ Configuração carregada com sucesso!")

In [ ]:
# Importar o Compliance Agent e os estados
from src.agent import ComplianceAgent
from src.states.idle import IdleState

# Inicializar o cliente OpenAI
client = OpenAI(api_key=api_key)

# Criar o agente com IdleState como estado inicial
agent = ComplianceAgent(client=client, initial_state=IdleState())

print(f"✅ Compliance Agent inicializado!")
print(f"📊 Estado atual: {agent.get_current_state_name()}")

## 🚀 Exemplo Básico de Uso {#exemplo-basico}

Vamos executar um exemplo completo do fluxo do agente e ver **exatamente** o que acontece:

### O que vamos ver:
1. **IdleState**: Captura a query do usuário
2. **AnalysisState**: LLM analisa e decide chamar a tool de auditoria
3. **AuditState**: Executa o ISR audit e retorna métricas (ISR, B2T, Delta, etc.)
4. **FinalResponseState**: LLM gera resposta final baseada no resultado da auditoria
5. **Retorno ao IdleState**: Agente fica pronto para a próxima query

### Por que retornar ao IdleState?
- ✅ Permite processar múltiplas queries sequencialmente
- ✅ O agente fica em estado "limpo" e pronto para nova entrada
- ✅ Cada ciclo completo (Idle → Analysis → Audit → FinalResponse → Idle) processa uma query

### 📌 Entendendo o Ciclo Completo

**O que acontece quando você executa uma query:**

1. **Você define a query**: `agent.current_query = "sua pergunta"`
2. **IdleState** captura a query e adiciona ao histórico
3. **AnalysisState** envia para o LLM, que decide chamar a tool `audit`
4. **AuditState** executa o ISR audit e retorna:
   - `decision`: "APROVADO" ou "BLOQUEADO"
   - `metrics`: ISR, B2T, Delta, JS_Bound, etc.
   - `reason`: Explicação da decisão
5. **FinalResponseState** gera resposta final ao usuário baseada no resultado da auditoria
6. **Retorno ao IdleState**: Agente fica pronto para a próxima query

**Resultados que você verá:**
- ✅ A resposta final do agente (texto gerado pelo LLM)
- ✅ As métricas da auditoria (ISR, B2T, Delta, etc.)
- ✅ A decisão da auditoria (APROVADO/BLOQUEADO)
- ✅ O estado atual do agente (deve ser IdleState ao final)

In [ ]:
import json
from src.states.idle import IdleState

# Exemplo 1: Query simples que requer auditoria
query = "Devo aprovar o pedido de crédito do cliente João Silva?"

# Criar um novo agente para este exemplo
agent_example = ComplianceAgent(client=client, initial_state=IdleState())

print("="*70)
print("🚀 EXEMPLO COMPLETO: PROCESSANDO UMA QUERY")
print("="*70)
print(f"\n📝 Query do usuário: {query}\n")

# Definir a query no agente
agent_example.current_query = query

# Executar o state machine até retornar ao IdleState
max_iterations = 20
iteration = 0

print("🔄 EXECUTANDO STATE MACHINE...\n")

while not isinstance(agent_example._state, IdleState) and iteration < max_iterations:
    state_name = agent_example.get_current_state_name()
    print(f"[Iteração {iteration + 1}] Estado atual: {state_name}")
    
    # Executar o estado atual
    agent_example.run()
    
    # Mostrar informações específicas de cada estado
    if state_name == "IdleState":
        print(f"   ✅ Query adicionada ao histórico")
        print(f"   📊 Próximo estado: {agent_example.get_current_state_name()}")
    
    elif state_name == "AnalysisState":
        print(f"   ✅ LLM analisou a query")
        if hasattr(agent_example, 'current_tool_call') and agent_example.current_tool_call:
            print(f"   🔧 Tool chamada: {agent_example.current_tool_call.function.name}")
            try:
                args = json.loads(agent_example.current_tool_call.function.arguments)
                print(f"   📋 Decisão proposta: {args.get('proposed_decision', 'N/A')}")
            except:
                pass
        print(f"   📊 Próximo estado: {agent_example.get_current_state_name()}")
    
    elif state_name == "AuditState":
        print(f"   ✅ ISR Audit executado")
        if agent_example.audit_result:
            audit = agent_example.audit_result
            decision = audit.get("decision", "UNKNOWN")
            metrics = audit.get("metrics", {})
            print(f"   📊 DECISÃO DA AUDITORIA: {decision}")
            print(f"   📈 Métricas ISR:")
            print(f"      - ISR: {metrics.get('ISR', 'N/A')}")
            print(f"      - B2T: {metrics.get('B2T', 'N/A')}")
            print(f"      - Delta: {metrics.get('Delta', 'N/A')}")
            print(f"      - JS_Bound: {metrics.get('JS_Bound', 'N/A')}")
            print(f"      - P_Original: {metrics.get('P_Original', 'N/A')}")
            print(f"      - P_Min_Permutation: {metrics.get('P_Min_Permutation', 'N/A')}")
            print(f"   💡 Razão: {audit.get('reason', 'N/A')}")
        print(f"   📊 Próximo estado: {agent_example.get_current_state_name()}")
    
    elif state_name == "FinalResponseState":
        # A resposta final será mostrada pelo próprio estado, mas vamos capturá-la
        print(f"   ✅ Resposta final gerada")
        print(f"   📊 Próximo estado: {agent_example.get_current_state_name()}")
    
    iteration += 1
    print()

print("="*70)
print("✅ PROCESSO COMPLETO!\n")

# Mostrar a resposta final do agente
print("📋 RESPOSTA FINAL DO AGENTE:")
print("="*70)
if agent_example.history:
    # A última mensagem do assistente é a resposta final
    for msg in reversed(agent_example.history):
        if msg.get("role") == "assistant" and msg.get("content"):
            print(msg["content"])
            break
print("="*70)

# Mostrar resumo da auditoria
print("\n📊 RESUMO DA AUDITORIA:")
print("="*70)
if agent_example.audit_result:
    audit = agent_example.audit_result
    print(f"Decisão: {audit.get('decision', 'N/A')}")
    print(f"ISR: {audit.get('metrics', {}).get('ISR', 'N/A')}")
    print(f"Razão: {audit.get('reason', 'N/A')}")
else:
    print("⚠️ Nenhum resultado de auditoria disponível")
print("="*70)

# Explicar o retorno ao IdleState
print("\n💡 POR QUE RETORNAR AO IdleState?")
print("="*70)
print("✅ O agente retornou ao IdleState, o que significa:")
print("   - O ciclo completo foi executado com sucesso")
print("   - O agente está pronto para processar a PRÓXIMA query")
print("   - O histórico foi preservado (útil para conversas contextuais)")
print("   - Você pode chamar agent.current_query = 'nova query' e executar novamente")
print("="*70)

## 🔍 Entendendo o Fluxo de Estados {#fluxo-estados}

Vamos examinar cada estado em detalhes:

In [ ]:
# Vamos criar um novo agente para demonstrar o fluxo passo a passo
agent_demo = ComplianceAgent(client=client, initial_state=IdleState())

# Query de exemplo
query_demo = "O lucro da empresa foi de 50 milhões. Devo aprovar o investimento?"

print("🔍 DEMONSTRAÇÃO PASSO A PASSO DO FLUXO\n")
print("="*70)

# Passo 1: IdleState
print("\n📍 PASSO 1: IdleState")
print(f"   Estado atual: {agent_demo.get_current_state_name()}")
agent_demo.current_query = query_demo
agent_demo.run()
print(f"   ✅ Query adicionada ao histórico")
print(f"   📊 Estado após transição: {agent_demo.get_current_state_name()}")

# Passo 2: AnalysisState
print("\n📍 PASSO 2: AnalysisState")
print(f"   Estado atual: {agent_demo.get_current_state_name()}")
agent_demo.run()
print(f"   ✅ LLM analisou a query e chamou a tool de auditoria")
print(f"   📊 Estado após transição: {agent_demo.get_current_state_name()}")

# Passo 3: AuditState
print("\n📍 PASSO 3: AuditState")
print(f"   Estado atual: {agent_demo.get_current_state_name()}")
agent_demo.run()
print(f"   ✅ ISR audit executado (Delta, B2T, JS Bound calculados)")
print(f"   📊 Estado após transição: {agent_demo.get_current_state_name()}")

# Passo 4: FinalResponseState
print("\n📍 PASSO 4: FinalResponseState")
print(f"   Estado atual: {agent_demo.get_current_state_name()}")
agent_demo.run()
print(f"   ✅ Resposta final gerada")
print(f"   📊 Estado após transição: {agent_demo.get_current_state_name()}")

print("\n" + "="*70)
print("✅ Ciclo completo executado!")

## 📊 Interpretando Resultados do ISR {#interpretando-resultados}

O ISR (Information Sufficiency Ratio) retorna métricas importantes. Vamos entender o que cada uma significa:

In [ ]:
import json

# Vamos examinar um resultado de auditoria
# (Normalmente você acessaria isso via agent.audit_result após o AuditState)

# Exemplo de resultado típico do ISR audit
example_result = {
    "decision": "APROVADO",
    "metrics": {
        "ISR": 1.5234,
        "B2T": 0.2341,
        "Delta": 0.3567,
        "JS_Bound": 0.0123,
        "P_Original": 0.95,
        "P_Min_Permutation": 0.89
    },
    "reason": "ISR calculated: 1.5234 (Threshold >= 1.0)"
}

print("📊 INTERPRETAÇÃO DAS MÉTRICAS ISR\n")
print("="*70)
print(json.dumps(example_result, indent=2, ensure_ascii=False))
print("="*70)

print("\n🔍 EXPLICAÇÃO DAS MÉTRICAS:\n")

print("1. ISR (Information Sufficiency Ratio):")
print("   - Valor: Delta / B2T")
print("   - ISR >= 1.0 → APROVADO (temos informação suficiente)")
print("   - ISR < 1.0  → BLOQUEADO (falta informação)")
print(f"   - Neste exemplo: {example_result['metrics']['ISR']} → APROVADO ✅\n")

print("2. B2T (Bits-to-Trust):")
print("   - Mede quanta informação precisamos (KL Divergence)")
print("   - Quanto maior, mais informação é necessária")
print(f"   - Neste exemplo: {example_result['metrics']['B2T']}\n")

print("3. Delta (Information Gain):")
print("   - Mede quanta informação temos (ganho médio)")
print("   - Quanto maior, mais informação disponível")
print(f"   - Neste exemplo: {example_result['metrics']['Delta']}\n")

print("4. JS_Bound (Jensen-Shannon Bound):")
print("   - Mede instabilidade entre permutações")
print("   - Quanto maior, mais instável (variação alta)")
print(f"   - Neste exemplo: {example_result['metrics']['JS_Bound']} (baixa instabilidade) ✅\n")

print("5. P_Original:")
print("   - Probabilidade na permutação original")
print(f"   - Neste exemplo: {example_result['metrics']['P_Original']} (95% de confiança) ✅\n")

print("6. P_Min_Permutation:")
print("   - Menor probabilidade entre todas as permutações")
print("   - Se muito baixa (< 0.20), Hard Veto é ativado")
print(f"   - Neste exemplo: {example_result['metrics']['P_Min_Permutation']} (robusto) ✅\n")

In [ ]:
# Criar um novo agente para múltiplas queries
agent_multi = ComplianceAgent(client=client, initial_state=IdleState())

queries = [
    "Devo aprovar o empréstimo de R$ 100.000 para a empresa X?",
    "O cliente Y tem histórico de inadimplência. Devo bloquear?",
    "A proposta Z atende todos os critérios. Posso aprovar?"
]

print("🔄 PROCESSANDO MÚLTIPLAS QUERIES\n")
print("="*70)

for i, query in enumerate(queries, 1):
    print(f"\n📝 Query {i}/{len(queries)}: {query}")
    print("-" * 70)
    
    agent_multi.current_query = query
    
    # Executar até retornar ao Idle
    iteration = 0
    while not isinstance(agent_multi._state, IdleState) and iteration < 20:
        agent_multi.run()
        iteration += 1
    
    # Verificar resultado da auditoria
    if agent_multi.audit_result:
        decision = agent_multi.audit_result.get("decision", "UNKNOWN")
        isr = agent_multi.audit_result.get("metrics", {}).get("ISR", 0.0)
        print(f"✅ Resultado: {decision} (ISR: {isr})")
    
    print(f"✅ Query {i} processada. Agente pronto para próxima query.\n")

print("="*70)
print("✅ Todas as queries processadas!")

### Exemplo 3: Acessando o Histórico de Conversação

In [ ]:
# Examinar o histórico completo de uma conversa
agent_history = ComplianceAgent(client=client, initial_state=IdleState())

query_history = "Devo aprovar o pedido do cliente ABC?"
agent_history.current_query = query_history

# Executar o fluxo completo
iteration = 0
while not isinstance(agent_history._state, IdleState) and iteration < 20:
    agent_history.run()
    iteration += 1

print("📜 HISTÓRICO COMPLETO DA CONVERSAÇÃO\n")
print("="*70)

for i, msg in enumerate(agent_history.history, 1):
    role = msg.get("role", "unknown")
    content = msg.get("content", "")
    tool_calls = msg.get("tool_calls")
    tool_call_id = msg.get("tool_call_id")
    
    print(f"\n[{i}] {role.upper()}")
    if content:
        # Truncar conteúdo muito longo
        content_preview = content[:200] + "..." if len(content) > 200 else content
        print(f"    Conteúdo: {content_preview}")
    if tool_calls:
        print(f"    🔧 Tool Calls: {len(tool_calls)} chamada(s)")
        for tc in tool_calls:
            print(f"       - {tc.function.name}")
    if tool_call_id:
        print(f"    🆔 Tool Call ID: {tool_call_id}")
    print("-" * 70)

print(f"\n📊 Total de mensagens: {len(agent_history.history)}")

### Exemplo 4: Usando a Tool ISR Diretamente

Você também pode usar o `SemanticISRAuditorTool` diretamente, sem passar pelo agente:

In [ ]:
from src.tools.isr_auditor import SemanticISRAuditorTool

# Criar a tool diretamente
isr_tool = SemanticISRAuditorTool(
    client=client,
    model="gpt-4o-mini",
    target_confidence=0.95,
    num_permutations=6,
    clipping_b=12.0,
    hard_veto_threshold=0.20
)

# Executar auditoria diretamente
prompt_context = "O cliente tem score de crédito 750 e renda de R$ 50.000/mês"
proposed_decision = "APROVADO"

print("🔧 USANDO ISR TOOL DIRETAMENTE\n")
print("="*70)
print(f"Contexto: {prompt_context}")
print(f"Decisão proposta: {proposed_decision}\n")

result_json = isr_tool.audit(prompt_context, proposed_decision)
result = json.loads(result_json)

print("📊 RESULTADO DA AUDITORIA:")
print(json.dumps(result, indent=2, ensure_ascii=False))
print("="*70)

## 💡 Boas Práticas {#boas-praticas}

### 1. Tratamento de Erros

Sempre envolva a execução do agente em try/except:

In [ ]:
def safe_run_agent(agent, query, max_iterations=20):
    """
    Executa o agente de forma segura com tratamento de erros.
    """
    try:
        agent.current_query = query
        
        iteration = 0
        while not isinstance(agent._state, IdleState) and iteration < max_iterations:
            agent.run()
            iteration += 1
        
        if iteration >= max_iterations:
            print("⚠️ Aviso: Máximo de iterações atingido")
            agent.transition_to(IdleState())
        
        return True
    except Exception as e:
        print(f"❌ Erro ao executar agente: {e}")
        # Resetar para IdleState em caso de erro
        from src.states.idle import IdleState
        agent.transition_to(IdleState())
        return False

# Exemplo de uso seguro
agent_safe = ComplianceAgent(client=client, initial_state=IdleState())
success = safe_run_agent(agent_safe, "Exemplo de query segura")

if success:
    print("✅ Execução bem-sucedida!")
else:
    print("❌ Execução falhou, mas agente foi resetado.")

### 2. Configuração Personalizada do ISR Tool

Você pode personalizar os parâmetros do ISR audit:

In [ ]:
# Criar agente com configuração personalizada do ISR tool
from src.agent import ComplianceAgent
from src.tools.isr_auditor import SemanticISRAuditorTool
from src.states.idle import IdleState

# Criar tool personalizada
custom_tool = SemanticISRAuditorTool(
    client=client,
    model="gpt-4o-mini",
    target_confidence=0.98,  # Mais rigoroso (98% de confiança)
    num_permutations=10,     # Mais permutações = mais robusto (mas mais caro)
    clipping_b=15.0,         # Clipping mais alto
    hard_veto_threshold=0.15 # Hard veto mais sensível
)

# Criar agente e substituir a tool padrão
agent_custom = ComplianceAgent(client=client, initial_state=IdleState())
agent_custom.tool = custom_tool

print("✅ Agente configurado com ISR tool personalizada!")
print(f"   - Target Confidence: {custom_tool.target_confidence}")
print(f"   - Num Permutations: {custom_tool.num_permutations}")
print(f"   - Clipping B: {custom_tool.clipping_b}")
print(f"   - Hard Veto Threshold: {custom_tool.hard_veto_threshold}")

### 3. Monitoramento e Logging

Para produção, adicione logging adequado:

In [ ]:
import logging
from datetime import datetime

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("ComplianceAgent")

def run_agent_with_logging(agent, query):
    """
    Executa o agente com logging detalhado.
    """
    logger.info(f"Iniciando processamento da query: {query[:50]}...")
    start_time = datetime.now()
    
    agent.current_query = query
    
    iteration = 0
    states_visited = []
    
    while not isinstance(agent._state, IdleState) and iteration < 20:
        state_name = agent.get_current_state_name()
        states_visited.append(state_name)
        logger.info(f"Estado atual: {state_name}")
        
        agent.run()
        iteration += 1
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    
    logger.info(f"Processamento completo em {duration:.2f}s")
    logger.info(f"Estados visitados: {' -> '.join(states_visited)}")
    
    if agent.audit_result:
        decision = agent.audit_result.get("decision", "UNKNOWN")
        isr = agent.audit_result.get("metrics", {}).get("ISR", 0.0)
        logger.info(f"Resultado: {decision} (ISR: {isr})")
    
    return agent

# Exemplo de uso com logging
agent_logged = ComplianceAgent(client=client, initial_state=IdleState())
run_agent_with_logging(agent_logged, "Exemplo com logging")

## 📚 Referências e Recursos

### Arquitetura
- **State Design Pattern**: [Gang of Four - Design Patterns](https://refactoring.guru/design-patterns/state)
- **ISR v3**: Baseado em "Predictable Compression Failures: Why Language Models Actually Hallucinate" (Chlon et al., 2025)

### Componentes do ISR v3
1. **Laplace Smoothing (PROB_FLOOR)**: Previne divisão por zero
2. **One-Sided Clipping (CLIPPING_B)**: Foca em perda de informação
3. **Hard Veto**: Detecta instabilidade severa
4. **Success Shortcut**: Bypass para alta confiança

### Métricas
- **ISR**: Information Sufficiency Ratio (Delta / B2T)
- **B2T**: Bits-to-Trust (KL Divergence)
- **Delta**: Information Gain médio
- **JS Bound**: Instabilidade medida via Jensen-Shannon

---

## 🎓 Conclusão

Este guia demonstrou:
- ✅ Como usar o Compliance Agent com State Pattern
- ✅ Como interpretar resultados do ISR
- ✅ Boas práticas para uso em produção
- ✅ Exemplos práticos de uso

**Próximos passos:**
1. Integre o agente em seu sistema
2. Configure logging adequado
3. Ajuste parâmetros do ISR conforme necessário
4. Monitore métricas em produção

**Dúvidas?** Consulte o código-fonte em `src/` ou entre em contato com a equipe de desenvolvimento.